In [ ]:
import zipfile
import PIL
import os
import matplotlib.pyplot as plt
import json
import csv
import tensorflow as tf
import shutil
import pandas as pd
import numpy as np

In [ ]:
f = open("../input/cassava-leaf-disease-classification/label_num_to_disease_map.json","r")
disease_names = json.load(f)
disease_names

In [ ]:
%rm -rf ./train_images
shutil.copytree("../input/cassava-leaf-disease-classification/train_images","./train_images")

In [ ]:
train_img_path = "./train_images/"
train_img_list = os.listdir(train_img_path)

In [ ]:
train_dict={}
with open("../input/cassava-leaf-disease-classification/train.csv") as file:
  reader = csv.reader(file,skipinitialspace=True)
  next(reader)
  for row in reader:
    train_dict[row[0]] = row[1]

In [ ]:
for i in range(5):
  path = train_img_path+str(i)
  if os.path.isdir(path) != True:
    os.mkdir(path)

In [ ]:
for name in train_img_list:
  path = train_img_path+name
  if os.path.isdir(path) != True:
    final_path = train_img_path+train_dict[name]
    shutil.move(path,final_path)
os.listdir(train_img_path)

In [ ]:
height,width = 256,256
batch_size=32
STEPS_PER_EPOCH = len(train_img_list)*0.8 / batch_size
VALIDATION_STEPS = len(train_img_list)*0.2 / batch_size
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        validation_split = 0.2,
        rotation_range=25,
        fill_mode="nearest",
        height_shift_range = 0.15,
        width_shift_range = 0.15,
)

train_generator = train_datagen.flow_from_directory(
    "./train_images",
    target_size = (height,width),
    batch_size=batch_size,
    class_mode = "sparse",
    shuffle = True,
    subset = "training"
)


validation_generator = train_datagen.flow_from_directory(
    "./train_images",
    target_size = (height,width),
    batch_size=batch_size,
    class_mode = "sparse",
    shuffle = True,
    subset = "validation"
)

In [ ]:
with tf.device('/device:GPU:0'):
    xception = tf.keras.applications.Xception(include_top = False, weights = None,
                               input_shape = (height,width, 3))
    model = xception.output
    model = tf.keras.layers.GlobalAveragePooling2D()(model)
    model = tf.keras.layers.Dense(5, activation = "softmax")(model)
    model = tf.keras.models.Model(xception.input, model)

    model.compile(optimizer ="adam",
                  loss = "sparse_categorical_crossentropy",
                  metrics = ["accuracy"])
    
    
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath="./xception_best_model.h5",
        save_weights_only=False,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True
    )
    
    history = model.fit(train_generator,
                    epochs=30,
                    steps_per_epoch = STEPS_PER_EPOCH,
                    verbose=1,
                    validation_data = validation_generator,
                    validation_steps = VALIDATION_STEPS,
                    callbacks=[checkpoint]
                )

In [ ]:
model = tf.keras.models.load_model("./xception_best_model.h5")

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(len(acc))
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.title('Training and validation loss')

In [ ]:
path = "../input/cassava-leaf-disease-classification/test_images/"
test_file_list = os.listdir(path)
predictions=[]
for filename in test_file_list:
  img = tf.keras.preprocessing.image.load_img(
      path+filename,target_size = (height,width)
  )
  arr = tf.keras.preprocessing.image.img_to_array(img)
  arr = tf.expand_dims(arr/255.,0)
  predictions.append(np.argmax(model.predict(arr)[0]))

df = pd.DataFrame(zip(test_file_list,predictions),columns = ["image_id","label"])
df.to_csv("./submission.csv",index=False)

In [ ]:
shutil.rmtree("./train_images")
os.remove("./xception_best_model.h5")